In [76]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os
import time


from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

domain = "openai.com" # <- put your domain to be crawled
full_url = "https://punjab.gov.pk/" # <- put your domain to be crawled with https or http

# Create a class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

In [77]:
""" Function to get the hyperlinks from a URL
def get_hyperlinks(url):

    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML

        with urllib.request.urlopen(url) as response:
            

            # If the response is not HTML, return an empty list
            if not response.info.get('Content-Type').startswith("text/html"):
                    return []

            # Decode the HTML
            html = response.read.decode('utf-8')
    except Exception as e:
        print("its from the get_hyperlink function.")
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks """

' Function to get the hyperlinks from a URL\ndef get_hyperlinks(url):\n\n    # Try to open the URL and read the HTML\n    try:\n        # Open the URL and read the HTML\n\n        with urllib.request.urlopen(url) as response:\n            \n\n            # If the response is not HTML, return an empty list\n            if not response.info.get(\'Content-Type\').startswith("text/html"):\n                    return []\n\n            # Decode the HTML\n            html = response.read.decode(\'utf-8\')\n    except Exception as e:\n        print("its from the get_hyperlink function.")\n        print(e)\n        return []\n\n    # Create the HTML Parser and then Parse the HTML to get hyperlinks\n    parser = HyperlinkParser()\n    parser.feed(html)\n\n    return parser.hyperlinks '

In [78]:
def get_hyperlinks(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # If the response is not HTML, return an empty list
        if not response.headers['Content-Type'].startswith("text/html"):
            return []

        html = response.text  # Decode response content as text

    except requests.exceptions.RequestException as e:
        print("its from the get_hyperlink function.")
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

In [79]:
# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        if link == None:
            continue
        print("this is from the loop in get_domain_hyperlinks function:")
        print(link)
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))

In [80]:
""" def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        pattern = r'[<>:"/\\|?*]'
        useable_filename = re.sub(pattern, "_", url[8:])
        print(useable_filename)
        with open('text/'+local_domain+'/'+ useable_filename + ".txt", "w", encoding="UTF-8") as f:

            # Get the text from the URL using BeautifulSoup
            soup = BeautifulSoup(requests.get(url).text, "html.parser")


            # Get the text but remove the tags
            text = soup.get_text()
            print(text)
    

            # If the crawler gets to a page that requires JavaScript, it will stop the crawl
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")

            # Otherwise, write the text to the file in the text directory
            f.write(text)

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)
                print("this the links queue:::::::")
                print(seen)

crawl(full_url) """

' def crawl(url):\n    # Parse the URL and get the domain\n    local_domain = urlparse(url).netloc\n\n    # Create a queue to store the URLs to crawl\n    queue = deque([url])\n\n    # Create a set to store the URLs that have already been seen (no duplicates)\n    seen = set([url])\n\n    # Create a directory to store the text files\n    if not os.path.exists("text/"):\n            os.mkdir("text/")\n\n    if not os.path.exists("text/"+local_domain+"/"):\n            os.mkdir("text/" + local_domain + "/")\n\n    # Create a directory to store the csv files\n    if not os.path.exists("processed"):\n            os.mkdir("processed")\n\n    # While the queue is not empty, continue crawling\n    while queue:\n\n        # Get the next URL from the queue\n        url = queue.pop()\n        print(url) # for debugging and to see the progress\n\n        # Save text from the url to a <url>.txt file\n        pattern = r\'[<>:"/\\|?*]\'\n        useable_filename = re.sub(pattern, "_", url[8:])\n   

In [82]:
def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        pattern = r'[<>:"/\\|?*]'
        useable_filename = re.sub(pattern, "_", url[8:])
        print(useable_filename)
        with open('text/'+local_domain+'/'+ useable_filename + ".txt", "w", encoding="UTF-8") as f:


            # Get the text from the URL using BeautifulSoup
            soup = BeautifulSoup(requests.get(url).text, "html.parser")


            # Get the text but remove the tags
            text = soup.get_text()
            #print(text)
            if ("Please turn JavaScript on and reload the page" in text):
                 

                options = Options()
                options.headless = True
                options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

                # Initialize the WebDriver
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

                try:
                    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
                    driver.get(url)
                    time.sleep(10)
                    # Use BeautifulSoup to parse the page source
                    soup = BeautifulSoup(driver.page_source, "html.parser")

                    # Extract text and remove tags
                    text = soup.get_text()
                    print(text)

                    
                except Exception as e:
                    print(f"Error while crawling {url}: {e}")
                finally:
                    driver.quit()



            # Otherwise, write the text to the file in the text directory
            f.write(text)

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)
                print("this the links queue:::::::")
                print(seen)

crawl(full_url)

https://punjab.gov.pk/
punjab.gov.pk_


SSLError: HTTPSConnectionPool(host='punjab.gov.pk', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))